In [1]:
from astropy.io import fits
import numpy as np
import tensorflow as tf

In [2]:
!ls scratch/spiel_test_var_psf/fits/400*

scratch/spiel_test_var_psf/fits/400-inputs.fits
scratch/spiel_test_var_psf/fits/400-outputs.fits
scratch/spiel_test_var_psf/fits/400-residuals.fits
scratch/spiel_test_var_psf/fits/400-targets.fits


In [3]:
var_psf = !ls scratch/spiel_test_var_psf/fits/*-residuals.fits
noscale = !ls scratch/spiel_test_noscale/fits/*-residuals.fits
notan = !ls scratch/spiel_test_var_psf_notan/fits/*-residuals.fits
multiscale = !ls scratch/spiel_test_multiscale/fits/*-residuals.fits
hdr = !ls scratch/spiel_test_hdr/fits/*-residuals.fits

In [4]:
rms = lambda x: np.sum(np.square(x))

In [5]:
rms_var_psf = [rms(fits.open(i)[0].data) for i in var_psf]
rms_noscale = [rms(fits.open(i)[0].data) for i in noscale]
rms_notan = [rms(fits.open(i)[0].data) for i in notan]
rms_multiscale = [rms(fits.open(i)[0].data) for i in multiscale]
rms_hdr = [rms(fits.open(i)[0].data) for i in hdr]

In [6]:
[np.average(x) for x in [rms_var_psf, rms_noscale, rms_notan, rms_multiscale, rms_hdr]]

[11.673036, 12.418399, 10.456104, 5.7386956, 10.591216]

In [7]:
!ls /home/gijs/Work/spiel/results/

0-skymodel.fits		 1-wsclean-image.fits	  2-wsclean-residual.fits
0-skymodel.txt		 1-wsclean-model.fits	  3-skymodel.fits
0-wsclean-dirty.fits	 1-wsclean-psf.fits	  3-skymodel.txt
0-wsclean-image.fits	 1-wsclean-residual.fits  3-wsclean-dirty.fits
0-wsclean-model.fits	 2-skymodel.fits	  3-wsclean-image.fits
0-wsclean-psf.fits	 2-skymodel.txt		  3-wsclean-model.fits
0-wsclean-residual.fits  2-wsclean-dirty.fits	  3-wsclean-psf.fits
1-skymodel.fits		 2-wsclean-image.fits	  3-wsclean-residual.fits
1-skymodel.txt		 2-wsclean-model.fits
1-wsclean-dirty.fits	 2-wsclean-psf.fits


In [8]:
files = !ls /home/gijs/Work/spiel/results/*-skymodel.fits
[fits.open(i)[0].data.max() for i in files]

[1.03, 1.05, 0.0459, 0.785]

In [9]:
files = !ls /home/gijs/Work/spiel/results/*-wsclean-dirty.fits
[fits.open(i)[0].data.max() for i in files]

[0.93312705, 1.0144283, 0.043181516, 0.7501933]

In [10]:
hdr_train = "/home/gijs/Work/astro-pix2pix/datasets/spiel_hdr/test/400.npy"
target_np = np.load(hdr_train)[:, :512 // 2]
input_np = np.load(hdr_train)[:, 512 // 2:]

In [11]:
target_np.min(), target_np.max(), input_np.min(), input_np.max()

(0.0, 0.513, -0.086591, 0.50760436)

In [12]:
target_tf_fits_path = "/home/gijs/Work/astro-pix2pix/scratch/spiel_test_hdr/fits/400-targets.fits"
target_tf_fits = fits.open(target_tf_fits_path)[0].data
input_tf_fits = fits.open("/home/gijs/Work/astro-pix2pix/scratch/spiel_test_hdr/fits/400-inputs.fits")[0].data

In [13]:
target_tf_fits.min(), target_tf_fits.max(), input_tf_fits.min(), input_tf_fits.max()

(0.0, 2.12, -0.47427642, 2.2891319)

In [14]:
target_tf = np.load("/home/gijs/Work/astro-pix2pix/scratch/spiel_test_hdr/npys/400-targets.npy")
input_tf = np.load("/home/gijs/Work/astro-pix2pix/scratch/spiel_test_hdr/npys/400-inputs.npy")

In [15]:
target_tf.min(), target_tf.max(), input_tf.min(), input_tf.max()

(0.0, 2.12, -0.47427642, 2.2891319)

In [16]:
def npy_decode(content):
    def internal(data):
        img = np.load(BytesIO(data)).astype(np.float32)
        img = np.expand_dims(img, axis=3)
        return img
    return tf.py_func(internal, [content], tf.float32)

def npy_encode(image):
    def internal(data):
        buf = BytesIO()
        np.save(buf, data)
        return buf.getvalue()
    return tf.py_func(internal, [image], tf.string)

In [ ]:
roundtrip.eval()